In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
train_tweet=pd.read_csv(r'C:\Users\Harshita arora\Downloads\train_tweets.csv')
train_tweet.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
train_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [4]:
train_tweet['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [5]:
def drop_features(features,data):
    data.drop(features,inplace=True,axis=1)

In [6]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",tweet.lower()).split())

In [7]:
train_tweet['processed_tweets'] = train_tweet['tweet'].apply(process_tweet)

In [8]:
train_tweet.head(10)

,id,label,tweet,processed_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i cant use cause they d...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation
5,6,0,[2/2] huge fan fare and big talking before the...,22 huge fan fare and big talking before they l...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land allin cavs champions clev...
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here im its so gr8


In [9]:
drop_features(['id','tweet'],train_tweet)

In [10]:
train_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 2 columns):
label               31962 non-null int64
processed_tweets    31962 non-null object
dtypes: int64(1), object(1)
memory usage: 499.5+ KB


In [11]:
train_tweet.head()

,label,processed_tweets
0,0,when a father is dysfunctional and is so selfi...
1,0,thanks for lyft credit i cant use cause they d...
2,0,bihday your majesty
3,0,model i love u take with u all the time in ur
4,0,factsguide society now motivation


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(train_tweet["processed_tweets"],train_tweet["label"], test_size = 0.2, random_state = 3)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [15]:
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [16]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [17]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [18]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(25569, 35561)
(25569, 35561)


In [19]:
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(6393, 35561)
(6393, 35561)


# Decision tree classifier

In [20]:
from sklearn.tree import DecisionTreeClassifier#model1
m1=DecisionTreeClassifier()
m1.fit(x_train_tfidf,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [21]:
prediction=m1.predict(x_test_tfidf)
print(prediction)

[0 0 0 ... 0 0 0]


In [22]:
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
c_m = confusion_matrix(y_test,prediction)
print(c_m)

[[5806  144]
 [ 175  268]]


In [23]:
accuracy_score(y_test,prediction)

0.9501016737056155

# Logistic Regression classifier

In [24]:
from sklearn.linear_model import LogisticRegression
m2=LogisticRegression()
m2.fit(x_train_tfidf,y_train)

C:\Users\Harshita arora\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
prediction2=m2.predict(x_test_tfidf)
print(prediction2)

[0 0 0 ... 0 0 0]


In [26]:
c_m=confusion_matrix(y_test,prediction2)
print(c_m)

[[5936   14]
 [ 312  131]]


In [27]:
accuracy_score(y_test,prediction2)

0.9490067261066791

# Support Vector Machine Classifier

In [28]:
from sklearn.svm import SVC#model3
m3=SVC()
m3.fit(x_train_tfidf,y_train)

C:\Users\Harshita arora\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [29]:
prediction3=m3.predict(x_test_tfidf)
print(prediction3)

[0 0 0 ... 0 0 0]


In [30]:
c_m=confusion_matrix(y_test,prediction3)
print(c_m)

[[5950    0]
 [ 443    0]]


In [31]:
accuracy_score(y_test,prediction3)

0.9307054590958861

# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier#model4
m4 = RandomForestClassifier(n_estimators=200)
m4.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
prediction4=m4.predict(x_test_tfidf)
print(prediction4)

[0 0 0 ... 0 0 0]


In [34]:
c_m=confusion_matrix(y_test,prediction4)
print(c_m)

[[5924   26]
 [ 206  237]]


In [35]:
accuracy_score(y_test,prediction4)

0.9637103081495385